In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# set version
version = 2
full = False

# load metrics
if full:
    metrics = pd.read_csv("lightning_logs/nc_128/metrics.csv")
else:
    metrics = pd.read_csv(f"lightning_logs/version_{version}/metrics.csv")

# merge entries on the same step
metrics = metrics.groupby("step").first()
display(metrics)

# drop nan values
no_nan = metrics.dropna()

In [ ]:
# plot NC1
nc_cols = no_nan.filter(like="nc_").columns
no_nan.plot(x="epoch", y=nc_cols)
plt.show()

In [ ]:
# plot train loss
train_cols = metrics.filter(like="train_").columns
metrics.plot(y=train_cols, use_index=True)
plt.ylim(0,1)
plt.xlabel("Step")
plt.ylabel("Cross-Entropy Loss & Accuracy")
plt.legend()
plt.show()

metrics.plot(y="lr", use_index=True)
plt.xlabel("Step")
plt.ylabel("Learning Rate")
plt.show()

In [ ]:
# extract DIB0
dib_0 = no_nan["dib_0"].iloc[0]
no_dib_0 = metrics.drop(columns="dib_0").dropna()

# plot DIB
dib_cols = no_dib_0.filter(like="dib_").columns
no_dib_0.plot(x="epoch", y=dib_cols)
plt.hlines(dib_0, 0, metrics.shape[0], color="black", label="dib_0")
# plt.xlim(0,150)
plt.legend()
plt.xlabel("Epoch")
plt.ylabel("Cross-Entropy Loss")
plt.show()

In [ ]:
# IP analysis
for dib in dib_cols:
    plt.scatter(metrics[dib], metrics["train_loss"], cmap="viridis", c=metrics["epoch"])
    plt.plot(metrics[dib], metrics["train_loss"], color="gray", linewidth=0.5)
plt.xlabel(r"$I_{\mathcal{V}}[Z \to \operatorname{Dec}(X,\mathcal{Y})]$")
plt.ylabel(r"$I_{\mathcal{V}}[Z \to Y]$")
plt.colorbar(label="Epochs")
plt.show()